In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Collecting

In [ ]:
# libraries
import pandas as pd
import numpy as np
#from numpy import genfromtxt
pd.options.mode.chained_assignment = None
from sklearn import preprocessing

In [ ]:
# Reading data - fetching TRAIN.csv data

#lenth_of_line = 2565 # can be automated
df_train_tmp = pd.read_csv("/kaggle/input/starcraft-2-player-prediction-challenge-2020/TRAIN.CSV", error_bad_lines=False, warn_bad_lines=False)
lenth_of_line = df_train_tmp.shape[1]
print ("lenth_of_line", lenth_of_line)


colnames = list()
colnames.append("id")
colnames.append("game_type")
for i in range (lenth_of_line-2):
    colnames.append("move"+str(i))
len (colnames)

df_train = pd.read_csv("/kaggle/input/starcraft-2-player-prediction-challenge-2020/TRAIN.CSV",names=colnames, header=None, error_bad_lines=False, warn_bad_lines=False)
n_training_games = df_train.shape[0]
print ("n_training_games", n_training_games)

In [ ]:
# Cleaning Training Data
#df_train.dropna(how='all')
df_train.dropna(subset=['move0'], inplace=True)
df_train.reset_index(drop=True, inplace = True)
df_train.tail(3)

In [ ]:
# Reading data - fetching TRAIN.csv data

lenth_of_line = 3446 # can be automated
#df_train_tmp = pd.read_csv("/kaggle/input/starcraft-2-player-prediction-challenge-2020/TEST.CSV", error_bad_lines=False, warn_bad_lines=False)
#lenth_of_line = df_train_tmp.shape[1]
print ("lenth_of_line", lenth_of_line)
colnames = list()
colnames.append("game_type")
for i in range (lenth_of_line-1):
    colnames.append("move"+str(i))
len (colnames)

df_test = pd.read_csv("/kaggle/input/starcraft-2-player-prediction-challenge-2020/TEST.CSV",names=colnames, header=None, error_bad_lines=False, warn_bad_lines=False)
n_testing_games = df_test.shape[0]
print ("n_testing_games", n_testing_games)

In [ ]:
df_test.tail()

# Data Visualisation

In [ ]:
#count number of players/labels in training dataset

list_unique_id = np.array(df_train["id"].unique())
print ("there are %d player labels in the training dataset"%len(list_unique_id))

In [ ]:
# [old Code]

# Adding first feature = type of game played
 
# For each id count how many game types he is associated to.
number_players = len(list_unique_id)
game_types = list()
for player_id in list_unique_id:
    
    player_history = df_train[df_train["id"]== player_id]
    game_types.append(len (player_history["game_type"].unique()))
avg = sum(game_types)/number_players
print("average number of types of games played by a player = " +  str(avg))
print(game_types)
print("This shows that a player only plays on one type of game")

In [ ]:
# get information on the length of games

moves_sec = list()
#df_train = df_train.head()

data = df_train.head(1000)
total_moves_list = []
game_times = []
lenth_of_line = df_train.shape[1]
for player_game_index in range (data.shape[0]):
    total_moves = 0
    game_time = 0
    for move_number in range (lenth_of_line-2):
        move = data.iloc[player_game_index]["move"+str(move_number)] #
        #print (move)
        if (not pd.isnull(move)):
            if move[:1] == "t":
                #game_time = int(move[1:])
                game_time += 5 
            else:
                total_moves +=1
    game_time += 0
    game_times.append(game_time)
    total_moves_list.append(total_moves)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(1)
plt.title("hist of game time")
plt.hist(game_times, bins=20)
plt.figure(2)
plt.title("hist of number of moves per game")
plt.hist(total_moves_list, bins=20)
#game_times
limit =50
short_games = [time for time in game_times if time<limit]
print ("Only", len(short_games), "games finish before",limit,"seconds")

In [ ]:
# RACE REPARTITION
races = df_train['game_type'].value_counts()
print(races)
plt = races.plot.bar()

In [ ]:
# number of games per player 
player_counts = df_train['id'].value_counts()
print(player_counts.describe())

In [ ]:
# list of unique actions
def find_unique_actions(df):
    unique_actions = set()
    actions = df.shape[0]
    for game_index in range(actions):
        for a in df.iloc[game_index]:
            if not pd.isnull(a) and a[0]!='t':
                unique_actions.add(a)
    return unique_actions

unique_hotkeys = sorted(list(find_unique_actions(df_train.drop(['id','game_type'], inplace=False, axis=1))))
print(unique_hotkeys)

keys = list(set([hotkey[:len(hotkey)-1] for hotkey in unique_hotkeys if 'hotkey' in hotkey]))
print(keys)

# Data Mining
## For each dataPoint we will extract features to build the input dataframe for our graph 

In [ ]:
############################# Accuracy 0.74 ####
# generate_features
# input: df_features = empty or with id, df = df_train or df_test
# output: df_features
#############################
 
# Loop through each of the training data set
def generate_features(df):
    
    if 'id' in df:
        df_features = pd.DataFrame(df['id'])
    else:
        df_features = pd.DataFrame()
        
    n_games = df.shape[0]
    lenth_of_line = df.shape[1]
    print ("n_games",n_games)
    df_features["plays_Terran"] = np.zeros(n_games)
    df_features["plays_Protoss"] = np.zeros(n_games)
    df_features["plays_Zerg"] = np.zeros(n_games)
    #df_features["game_time"] = np.zeros(n_games) 
    df_features["moves_per_sec"] = np.zeros(n_games)
    #df_features["n_moves"] = np.zeros(n_games) 
    df_features["n_hotkeys"] = np.zeros(n_games)
    df_features["n_base"] = np.zeros(n_games) 
    df_features["n_single_mineral"] = np.zeros(n_games)
    df_features["s"] = np.zeros(n_games)
    
    #do for all hot keys
    for i in range (9):
        for j in range (3):
            col = "hotkey" + str(i+1) + str(j)
            df_features[col] = np.zeros(n_games)  
            
    for game_index in range (n_games):
        if game_index%20 == 0:
            print (game_index)
        #game type
        game_type = df["game_type"][game_index]
        if "Terran" == game_type:
            df_features["plays_Terran"][game_index] = 1
        if "Protoss" == game_type:
            df_features["plays_Protoss"][game_index] = 1
        if "Zerg" == game_type:
            df_features["plays_Zerg"][game_index] = 1

        #Looping through the moves of the game
        total_moves = 0
        game_time = 0

        n_hotkeys = 0
        n_s = 0
        n_base = 0
        n_single_mineral = 0
        dict_hotkey = {}


        for move_number in range (lenth_of_line-2):
            move = df.iloc[game_index]["move"+str(move_number)] 
            if (not pd.isnull(move)):
                if move[:1] == "t":
                    game_time = int(move[1:])
                else:
                    total_moves +=1
                    if move[:4]=="Base":
                        n_base +=1
                    if move=="s":
                        n_s +=1
                    if move=="SingleMineral":
                        n_single_mineral +=1
                    if move[:6] == "hotkey":
                        n_hotkeys +=1
                        key = move 
                        if key in dict_hotkey.keys():
                            dict_hotkey[key] +=1
                        else:
                            dict_hotkey[key] =1
                        
        game_time +=5
        df_features["moves_per_sec"][game_index] = total_moves/game_time            
        #df_features["game_time"][game_index] = game_time

        #df_features["n_moves"][game_index]  = total_moves
        df_features["n_hotkeys"][game_index]  = (n_hotkeys+0.0001)/game_time
        df_features["n_base"][game_index]  = (n_base+0.0001)/game_time
        
        #df_features["n_single_mineral"] = (n_single_mineral+0.0001)/game_time
        df_features["n_single_mineral"] = (n_single_mineral)
        
        df_features["s"] = (n_s+0.0001)/game_time
        for i in range (9):
            for j in range (3):
                col = "hotkey" + str(i+1) + str(j)
                if col in dict_hotkey.keys():
                    df_features[col] = dict_hotkey[col]/game_time
                else:
                    df_features[col] = 0
        
    print ("Done") 
    return df_features
    


In [ ]:
############################# Accuracy 0.87 ###
# generate_features_2
# input: df = df_train or df_test
# output: df_features
#############################

hotkey_columns = ['first_hotkey','second_hotkey','third_hotkey','fourth_hotkey','fifth_hotkey','sixth_hotkey',
                  'seventh_hotkey','eighth_hotkey','nineth_hotkey']

#hotkey_columns = ['first_hotkey','second_hotkey','third_hotkey','fourth_hotkey','fifth_hotkey','sixth_hotkey','seventh_hotkey','eighth_hotkey']

def generate_features_2(df):
    
    le = preprocessing.LabelEncoder()
    if 'id' in df:
        df_features = pd.DataFrame(df['id'])
    else:
        df_features = pd.DataFrame()
    
    n_games = df.shape[0]
    lenth_of_line = df.shape[1]
    df_features["plays_Terran"] = np.zeros(n_games)
    df_features["plays_Protoss"] = np.zeros(n_games)
    df_features["plays_Zerg"] = np.zeros(n_games)
    features = []
    hotkeys = unique_hotkeys
    set_hotkeys = set(hotkeys)
    for game_index in range(n_games):
        if game_index%20 == 0:
            print (game_index)
        first_hotkeys = []
        #race = row["game_type"]
        #actions = row["actions"]
        hotkeys_count = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t10 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t20 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t30 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t60 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t90 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t120 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t150 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t180 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t210 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t250 = {hotkey:0 for hotkey in hotkeys}
        t10 = False
        t20 = False
        t30 = False
        t60 = False
        t90 = False
        t120 = False
        t150 = False
        t180 = False
        t210 = False
        t250 = False
        keys_count = {key:0 for key in keys}
        
        #game type
        game_type = df["game_type"][game_index]
        if "Terran" == game_type:
            df_features["plays_Terran"][game_index] = 1
        if "Protoss" == game_type:
            df_features["plays_Protoss"][game_index] = 1
        if "Zerg" == game_type:
            df_features["plays_Zerg"][game_index] = 1
        
        for move_number in range (lenth_of_line-2):
            action = df.iloc[game_index]["move"+str(move_number)]
            if action in set_hotkeys:
                hotkeys_count[action]+=1
                if not t10:
                    hotkeys_count_t10[action]+=1
                if not t20:
                    hotkeys_count_t20[action]+=1
                if not t30:
                    hotkeys_count_t30[action]+=1
                if not t60:
                    hotkeys_count_t60[action]+=1
                if not t90:
                    hotkeys_count_t90[action]+=1
                if not t120:
                    hotkeys_count_t120[action]+=1
                if not t150:
                    hotkeys_count_t150[action]+=1
                if not t180:
                    hotkeys_count_t180[action]+=1
                if not t210:
                    hotkeys_count_t210[action]+=1
                if not t250:
                    hotkeys_count_t250[action]+=1
            #print (action)
            if not pd.isnull(action):
                if 'hotkey' in action:
                    keys_count[action[:len(action)-1]]+=1
                if len(first_hotkeys)<len(hotkey_columns) and 'hotkey' in action:
                    first_hotkeys.append(action)
                elif 't10' in action:
                    t10 = True
                elif 't20' in action:
                    t20 = True
                elif 't30' in action:
                    t30 = True
                elif 't60' in action:
                    t60 = True
                elif 't90' in action:
                    t90 = True
                elif 't120' in action:
                    t120 = True
                elif 't150' in action:
                    t150 = True
                elif 't180' in action:
                    t180 = True
                elif 't210' in action:
                    t210 = True
                elif 't250' in action:
                    t250 = True
                
        while len(first_hotkeys)<len(hotkey_columns):
            first_hotkeys.append('None')
        current = [*[hotkeys_count[hotkey] for hotkey in hotkeys],*[hotkeys_count_t10[hotkey] for hotkey in hotkeys],*[hotkeys_count_t20[hotkey] for hotkey in hotkeys],*[hotkeys_count_t30[hotkey] for hotkey in hotkeys],*[hotkeys_count_t60[hotkey] for hotkey in hotkeys],*[hotkeys_count_t90[hotkey] for hotkey in hotkeys],*[hotkeys_count_t120[hotkey] for hotkey in hotkeys],*[hotkeys_count_t150[hotkey] for hotkey in hotkeys], *[hotkeys_count_t180[hotkey] for hotkey in hotkeys],*[hotkeys_count_t210[hotkey] for hotkey in hotkeys],*[hotkeys_count_t250[hotkey] for hotkey in hotkeys],*[keys_count[key] for key in keys], *first_hotkeys]
        
        features.append(current)
        #print("len(features[-1])",len(features[-1]))
    new_df = pd.DataFrame(features, columns=[*hotkeys,*[hotkey+'_t10' for hotkey in hotkeys],*[hotkey+'_t20' for hotkey in hotkeys],*[hotkey+'_t30' for hotkey in hotkeys],*[hotkey+'_t60' for hotkey in hotkeys],*[hotkey+'_t90' for hotkey in hotkeys],*[hotkey+'_t120' for hotkey in hotkeys],*[hotkey+'_t150' for hotkey in hotkeys], *[hotkey+'_t180' for hotkey in hotkeys],*[hotkey+'_t210' for hotkey in hotkeys], *[hotkey+'_t250' for hotkey in hotkeys], *keys, *hotkey_columns])
    
    # label encoder for the first attributes
    for col in hotkey_columns:
        new_df[col] = le.fit_transform(new_df[col])
    
    df_features = pd.concat([df_features, new_df], axis=1, sort=False)
    return df_features

In [ ]:
############################# Accuracy 0.932 ###
# generate_features_3
# input: df = df_train or df_test
# output: df_features
#############################
# Add more hot key columns embedding #done
# Add stps 5 by 5 for the first 30 seconds # done
# Remove all the games with less than 5 seconds? # This should be done before or after the feature creation

hotkey_columns = ['first_hotkey','second_hotkey','third_hotkey','fourth_hotkey','fifth_hotkey','sixth_hotkey',
                  'seventh_hotkey','eighth_hotkey','nineth_hotkey','tenth_hotkey', 'eleventh_hotkey', 'twelfth_hotkey',
                  'thirteenth_hotkey','forteeth_hotkey','fifteenth_hotkey']

def generate_features_3(df):
    
    le = preprocessing.LabelEncoder()
    
    #code to work for test and training data
    if 'id' in df:
        df_features = pd.DataFrame(df['id'])
    else:
        df_features = pd.DataFrame()

    n_games = df.shape[0]
    lenth_of_line = df.shape[1]
    df_features["plays_Terran"] = np.zeros(n_games)
    df_features["plays_Protoss"] = np.zeros(n_games)
    df_features["plays_Zerg"] = np.zeros(n_games)
    features = []
    hotkeys = unique_hotkeys
    set_hotkeys = set(hotkeys)
    
    for game_index in range(n_games):
        if game_index%20 == 0: # progress counter
            print (game_index)
        first_hotkeys = []
        hotkeys_count = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t5 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t10 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t15 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t20 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t25 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t30 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t60 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t90 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t120 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t150 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t180 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t210 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t250 = {hotkey:0 for hotkey in hotkeys}
        t5 = False
        t10 = False
        t15 = False
        t20 = False
        t25 = False
        t30 = False
        t60 = False
        t90 = False
        t120 = False
        t150 = False
        t180 = False
        t210 = False
        t250 = False
        keys_count = {key:0 for key in keys}
        
        #game type
        game_type = df["game_type"][game_index]
        if "Terran" == game_type:
            df_features["plays_Terran"][game_index] = 1
        if "Protoss" == game_type:
            df_features["plays_Protoss"][game_index] = 1
        if "Zerg" == game_type:
            df_features["plays_Zerg"][game_index] = 1
        
        for move_number in range (lenth_of_line-2):
            action = df.iloc[game_index]["move"+str(move_number)]
            if action in set_hotkeys:
                hotkeys_count[action]+=1
                if not t5:
                    hotkeys_count_t5[action]+=1
                if not t10:
                    hotkeys_count_t10[action]+=1
                if not t15:
                    hotkeys_count_t15[action]+=1
                if not t20:
                    hotkeys_count_t20[action]+=1
                if not t25:
                    hotkeys_count_t25[action]+=1
                if not t30:
                    hotkeys_count_t30[action]+=1
                if not t60:
                    hotkeys_count_t60[action]+=1
                if not t90:
                    hotkeys_count_t90[action]+=1
                if not t120:
                    hotkeys_count_t120[action]+=1
                if not t150:
                    hotkeys_count_t150[action]+=1
                if not t180:
                    hotkeys_count_t180[action]+=1
                if not t210:
                    hotkeys_count_t210[action]+=1
                if not t250:
                    hotkeys_count_t250[action]+=1
                    
            if not pd.isnull(action):
                if 'hotkey' in action:
                    keys_count[action[:len(action)-1]]+=1
                if len(first_hotkeys)<len(hotkey_columns) and 'hotkey' in action:
                    first_hotkeys.append(action)
                if 't5' in action:
                    t5 = True
                elif 't10' in action:
                    t10 = True
                elif 't15' in action:
                    t15 = True
                elif 't20' in action:
                    t20 = True
                elif 't25' in action:
                    t25 = True
                elif 't30' in action:
                    t30 = True
                elif 't60' in action:
                    t60 = True
                elif 't90' in action:
                    t90 = True
                elif 't120' in action:
                    t120 = True
                elif 't150' in action:
                    t150 = True
                elif 't180' in action:
                    t180 = True
                elif 't210' in action:
                    t210 = True
                elif 't250' in action:
                    t250 = True
                
        while len(first_hotkeys)<len(hotkey_columns):
            first_hotkeys.append('None')
        current = [*[hotkeys_count[hotkey] for hotkey in hotkeys],*[hotkeys_count_t5[hotkey] for hotkey in hotkeys],*[hotkeys_count_t10[hotkey] for hotkey in hotkeys],*[hotkeys_count_t15[hotkey] for hotkey in hotkeys],*[hotkeys_count_t20[hotkey] for hotkey in hotkeys],*[hotkeys_count_t25[hotkey] for hotkey in hotkeys],*[hotkeys_count_t30[hotkey] for hotkey in hotkeys],*[hotkeys_count_t60[hotkey] for hotkey in hotkeys],*[hotkeys_count_t90[hotkey] for hotkey in hotkeys],*[hotkeys_count_t120[hotkey] for hotkey in hotkeys],*[hotkeys_count_t150[hotkey] for hotkey in hotkeys], *[hotkeys_count_t180[hotkey] for hotkey in hotkeys],*[hotkeys_count_t210[hotkey] for hotkey in hotkeys],*[hotkeys_count_t250[hotkey] for hotkey in hotkeys],*[keys_count[key] for key in keys], *first_hotkeys]
        features.append(current)
        #print("len(features[-1])",len(features[-1]))
    new_df = pd.DataFrame(features, columns=[*hotkeys,*[hotkey+'_t5' for hotkey in hotkeys],*[hotkey+'_t10' for hotkey in hotkeys],*[hotkey+'_t15' for hotkey in hotkeys],*[hotkey+'_t20' for hotkey in hotkeys],*[hotkey+'_t25' for hotkey in hotkeys],*[hotkey+'_t30' for hotkey in hotkeys],*[hotkey+'_t60' for hotkey in hotkeys],*[hotkey+'_t90' for hotkey in hotkeys],*[hotkey+'_t120' for hotkey in hotkeys],*[hotkey+'_t150' for hotkey in hotkeys], *[hotkey+'_t180' for hotkey in hotkeys],*[hotkey+'_t210' for hotkey in hotkeys], *[hotkey+'_t250' for hotkey in hotkeys], *keys, *hotkey_columns])
    
    # label encoder for the first attributes
    for col in hotkey_columns:
        new_df[col] = le.fit_transform(new_df[col])
    
    df_features = pd.concat([df_features, new_df], axis=1, sort=False)
    return df_features

In [ ]:
############################# Accuracy 0.X ###
# generate_features_4
# input: df = df_train or df_test
# output: df_features
#############################
# Add more hot key columns embedding #done
# Add stps 5 by 5 for the first 30 seconds # done
# Remove all the games with less than 5 seconds? # This should be done before or after the feature creation

hotkey_columns = ['first_hotkey','second_hotkey','third_hotkey','fourth_hotkey','fifth_hotkey','sixth_hotkey',
                  'seventh_hotkey','eighth_hotkey','nineth_hotkey','tenth_hotkey', 'eleventh_hotkey', 'twelfth_hotkey',
                  'thirteenth_hotkey','forteeth_hotkey','fifteenth_hotkey','sixteenth','seventeenth','eighteenth','nineteenth','twenteeth']

def generate_features_4(df):
    
    le = preprocessing.LabelEncoder()
    
    #code to work for test and training data
    if 'id' in df:
        df_features = pd.DataFrame(df['id'])
    else:
        df_features = pd.DataFrame()

    n_games = df.shape[0]
    lenth_of_line = df.shape[1]
    df_features["plays_Terran"] = np.zeros(n_games)
    df_features["plays_Protoss"] = np.zeros(n_games)
    df_features["plays_Zerg"] = np.zeros(n_games)
    features = []
    hotkeys = unique_hotkeys
    set_hotkeys = set(hotkeys)
    
    for game_index in range(n_games):
        if game_index%20 == 0: # progress counter
            print (game_index)
        first_hotkeys = []
        hotkeys_count = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t5 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t10 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t15 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t20 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t25 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t30 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t60 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t90 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t120 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t150 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t180 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t210 = {hotkey:0 for hotkey in hotkeys}
        hotkeys_count_t250 = {hotkey:0 for hotkey in hotkeys}
        t5 = False
        t10 = False
        t15 = False
        t20 = False
        t25 = False
        t30 = False
        t60 = False
        t90 = False
        t120 = False
        t150 = False
        t180 = False
        t210 = False
        t250 = False
        keys_count = {key:0 for key in keys}
        
        #game type
        game_type = df["game_type"][game_index]
        if "Terran" == game_type:
            df_features["plays_Terran"][game_index] = 1
        if "Protoss" == game_type:
            df_features["plays_Protoss"][game_index] = 1
        if "Zerg" == game_type:
            df_features["plays_Zerg"][game_index] = 1
        
        for move_number in range (lenth_of_line-2):
            action = df.iloc[game_index]["move"+str(move_number)]
            if action in set_hotkeys:
                hotkeys_count[action]+=1
                if not t5:
                    hotkeys_count_t5[action]+=1
                if not t10:
                    hotkeys_count_t10[action]+=1
                if not t15:
                    hotkeys_count_t15[action]+=1
                if not t20:
                    hotkeys_count_t20[action]+=1
                if not t25:
                    hotkeys_count_t25[action]+=1
                if not t30:
                    hotkeys_count_t30[action]+=1
                if not t60:
                    hotkeys_count_t60[action]+=1
                if not t90:
                    hotkeys_count_t90[action]+=1
                if not t120:
                    hotkeys_count_t120[action]+=1
                if not t150:
                    hotkeys_count_t150[action]+=1
                if not t180:
                    hotkeys_count_t180[action]+=1
                if not t210:
                    hotkeys_count_t210[action]+=1
                if not t250:
                    hotkeys_count_t250[action]+=1
                    
            if not pd.isnull(action):
                if 'hotkey' in action:
                    keys_count[action[:len(action)-1]]+=1
                if len(first_hotkeys)<len(hotkey_columns) and 'hotkey' in action:
                    first_hotkeys.append(action)
                if 't5' in action:
                    t5 = True
                elif 't10' in action:
                    t10 = True
                elif 't15' in action:
                    t15 = True
                elif 't20' in action:
                    t20 = True
                elif 't25' in action:
                    t25 = True
                elif 't30' in action:
                    t30 = True
                elif 't60' in action:
                    t60 = True
                elif 't90' in action:
                    t90 = True
                elif 't120' in action:
                    t120 = True
                elif 't150' in action:
                    t150 = True
                elif 't180' in action:
                    t180 = True
                elif 't210' in action:
                    t210 = True
                elif 't250' in action:
                    t250 = True
                
        while len(first_hotkeys)<len(hotkey_columns):
            first_hotkeys.append('None')
        current = [*[hotkeys_count[hotkey] for hotkey in hotkeys],*[hotkeys_count_t5[hotkey] for hotkey in hotkeys],*[hotkeys_count_t10[hotkey] for hotkey in hotkeys],*[hotkeys_count_t15[hotkey] for hotkey in hotkeys],*[hotkeys_count_t20[hotkey] for hotkey in hotkeys],*[hotkeys_count_t25[hotkey] for hotkey in hotkeys],*[hotkeys_count_t30[hotkey] for hotkey in hotkeys],*[hotkeys_count_t60[hotkey] for hotkey in hotkeys],*[hotkeys_count_t90[hotkey] for hotkey in hotkeys],*[hotkeys_count_t120[hotkey] for hotkey in hotkeys],*[hotkeys_count_t150[hotkey] for hotkey in hotkeys], *[hotkeys_count_t180[hotkey] for hotkey in hotkeys],*[hotkeys_count_t210[hotkey] for hotkey in hotkeys],*[hotkeys_count_t250[hotkey] for hotkey in hotkeys],*[keys_count[key] for key in keys], *first_hotkeys]
        features.append(current)
        #print("len(features[-1])",len(features[-1]))
    new_df = pd.DataFrame(features, columns=[*hotkeys,*[hotkey+'_t5' for hotkey in hotkeys],*[hotkey+'_t10' for hotkey in hotkeys],*[hotkey+'_t15' for hotkey in hotkeys],*[hotkey+'_t20' for hotkey in hotkeys],*[hotkey+'_t25' for hotkey in hotkeys],*[hotkey+'_t30' for hotkey in hotkeys],*[hotkey+'_t60' for hotkey in hotkeys],*[hotkey+'_t90' for hotkey in hotkeys],*[hotkey+'_t120' for hotkey in hotkeys],*[hotkey+'_t150' for hotkey in hotkeys], *[hotkey+'_t180' for hotkey in hotkeys],*[hotkey+'_t210' for hotkey in hotkeys], *[hotkey+'_t250' for hotkey in hotkeys], *keys, *hotkey_columns])
    
    # label encoder for the first attributes
    for col in hotkey_columns:
        new_df[col] = le.fit_transform(new_df[col])
    
    df_features = pd.concat([df_features, new_df], axis=1, sort=False)
    return df_features

In [ ]:
#15 min
#df_train_features = generate_features(df_train.head(50))
#df_train_features = generate_features_2(df_train)
#df_train_features = generate_features_3(df_train)
df_train_features = generate_features_4(df_train)

In [65]:
df_train_features.tail(3)

,id,plays_Terran,plays_Protoss,plays_Zerg,Base,SingleMineral,hotkey00,hotkey01,hotkey02,hotkey10,...,eleventh_hotkey,twelfth_hotkey,thirteenth_hotkey,forteeth_hotkey,fifteenth_hotkey,sixteenth,seventeenth,eighteenth,nineteenth,twenteeth
3041,http://xx.battle.net/sc2/en/profile/405/1/MMA/,1.0,0.0,0.0,0,5,5,0,56,32,...,7,7,7,7,6,6,6,6,7,6
3042,http://xx.battle.net/sc2/en/profile/410/1/STBo...,1.0,0.0,0.0,0,42,0,0,0,18,...,5,7,5,7,6,4,6,3,5,6
3043,http://xx.battle.net/sc2/en/profile/405/1/MMA/,1.0,0.0,0.0,0,2,1,0,0,13,...,7,7,7,7,6,6,6,6,10,6


In [ ]:
#df_test_features = generate_features(df_test)
#df_test_features = generate_features_2(df_test)
#df_test_features = generate_features_3(df_test)
df_test_features = generate_features_4(df_test)

In [66]:
df_test_features.tail(3)

,plays_Terran,plays_Protoss,plays_Zerg,Base,SingleMineral,hotkey00,hotkey01,hotkey02,hotkey10,hotkey11,...,eleventh_hotkey,twelfth_hotkey,thirteenth_hotkey,forteeth_hotkey,fifteenth_hotkey,sixteenth,seventeenth,eighteenth,nineteenth,twenteeth
337,1.0,0.0,0.0,0,27,0,0,0,24,0,...,6,5,5,4,5,5,4,5,3,3
338,1.0,0.0,0.0,0,1,1,0,0,7,0,...,6,7,5,6,5,5,6,5,6,5
339,1.0,0.0,0.0,0,43,0,0,0,27,0,...,4,7,2,4,5,3,6,5,4,5


# Building the Model

In [67]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [68]:
print (len(df_train_features),len(df_test_features))

3044 340


In [69]:
len(list(df_train_features.columns[1:]))

495

In [70]:
feature_cols = list(df_train_features.columns[1:])
X = df_train_features[feature_cols] # Features
y = df_train_features.id # Target variable

In [71]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [72]:
# Create Decision Tree classifer object (SIMPLE)
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
# Predict the response for test dataset
y_pred = clf.predict(X_test)
# Model Accuracy, how often is the classifier correct? on built test set
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy train:",metrics.accuracy_score(y_train, clf.predict(X_train)))

Accuracy: 0.7210065645514223
Accuracy train: 1.0


In [74]:
def evaluate(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    #print("Accuracy:",accuracy)
    return accuracy

In [75]:
# Decision Tree
# Import necessary modules
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": randint(1, 20),
              "max_features": randint(1, 40),
              "min_samples_leaf": randint(1, 20),
              "criterion": ["gini", "entropy"]}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()
# Instantiate the RandomizedSearchCV object: tree_cv
dt_tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# Fit it to the data
dt_tree_cv.fit(X_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6824cb4d10>,
                                        'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6824cb4050>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f68251ded10>})

In [77]:
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(dt_tree_cv.best_params_))
print("Best score is {}".format(dt_tree_cv.best_score_))

best_model = dt_tree_cv.best_estimator_
accuracy = evaluate(best_model, X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Tuned Decision Tree Parameters: {'criterion': 'entropy', 'max_depth': 14, 'max_features': 30, 'min_samples_leaf': 1}
Best score is 0.8098591549295774
Accuracy: 76.59%


In [87]:
# Random Forest
# Import necessary modules, 50sec per iteration

# Setup the parameters and distributions to sample from: param_dist
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

# Instantiate a Decision Tree classifier: tree
tree =RandomForestClassifier()
# Instantiate the RandomizedSearchCV object: tree_cv
rf_tree_cv = RandomizedSearchCV(estimator = tree, param_distributions = random_grid, n_iter = 30, cv = 3, verbose=1, random_state=42)

# Fit it to the data
rf_tree_cv.fit(X_train,y_train)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 48.1min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=30,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=1)

In [88]:
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(rf_tree_cv.best_params_))
print("Best score is {}".format(rf_tree_cv.best_score_))

best_model = rf_tree_cv.best_estimator_
accuracy = evaluate(best_model, X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Tuned Decision Tree Parameters: {'n_estimators': 2000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 110, 'bootstrap': True}
Best score is 0.9276995305164318
Accuracy: 90.59%


In [96]:
# Testing
testing_model = RandomForestClassifier(n_estimators= 2000, min_samples_split= 2, min_samples_leaf= 2, max_features= "auto", max_depth= 200, bootstrap= False)
testing_model.fit(X_train,y_train)
accuracy = evaluate(testing_model, X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.70%


In [80]:
# XGBoost

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# fit model to training data
XGB_model = XGBClassifier()
XGB_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [81]:
# Print the tuned parameters and score
y_pred = XGB_model.predict(X_test)
# Model Accuracy, how often is the classifier correct? on built test set
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy train:",metrics.accuracy_score(y_train, clf.predict(X_train)))


Accuracy: 0.8588621444201313
Accuracy train: 1.0


In [84]:
XGB_model = XGBClassifier()
eval_set = [(X_test, y_test)]
XGB_model.fit(X_train, y_train,eval_set=eval_set)

[0]	validation_0-merror:0.75274
[1]	validation_0-merror:0.57221
[2]	validation_0-merror:0.45514
[3]	validation_0-merror:0.35449
[4]	validation_0-merror:0.30525
[5]	validation_0-merror:0.27790
[6]	validation_0-merror:0.26258
[7]	validation_0-merror:0.23742
[8]	validation_0-merror:0.21991
[9]	validation_0-merror:0.21007
[10]	validation_0-merror:0.20241
[11]	validation_0-merror:0.19256
[12]	validation_0-merror:0.18818
[13]	validation_0-merror:0.17943
[14]	validation_0-merror:0.17615
[15]	validation_0-merror:0.17177
[16]	validation_0-merror:0.16849
[17]	validation_0-merror:0.16302
[18]	validation_0-merror:0.16083
[19]	validation_0-merror:0.15427
[20]	validation_0-merror:0.15427
[21]	validation_0-merror:0.15208
[22]	validation_0-merror:0.15099
[23]	validation_0-merror:0.15099
[24]	validation_0-merror:0.15208
[25]	validation_0-merror:0.14770
[26]	validation_0-merror:0.14770
[27]	validation_0-merror:0.14333
[28]	validation_0-merror:0.14223
[29]	validation_0-merror:0.14442
[30]	validation_0-me

TypeError: type str doesn't define __round__ method

In [85]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 85.89%


## Evaluating the unlabeled testing set - Creating submission file

In [97]:
#final_model = DecisionTreeClassifier()
final_model = RandomForestClassifier(n_estimators= 2000, min_samples_split= 2, min_samples_leaf= 1, max_features= "auto", max_depth= 200, bootstrap= False)
#final_model = XGBClassifier()
final_model.fit(X,y)

# Predict the response for test dataset
#y_pred = clf.predict(X_test)
submission = final_model.predict(df_test_features[feature_cols])

In [99]:
file_name = "submission7"
df_submission = pd.DataFrame()
df_submission['RowId'] = range(1,len(submission)+1)
df_submission['prediction'] = submission
df_submission.to_csv('/kaggle/working/'+file_name+'.csv', index = False)
df_submission.head()

,RowId,prediction
0,1,http://eu.battle.net/sc2/en/profile/3538115/1/...
1,2,http://eu.battle.net/sc2/en/profile/2896854/1/...
2,3,http://eu.battle.net/sc2/en/profile/3973341/1/...
3,4,http://eu.battle.net/sc2/en/profile/250458/1/V...
4,5,http://eu.battle.net/sc2/en/profile/950504/1/G...


In [ ]:
df_submission.index